# DreamJobber

**Tech Edition**

---

**Process**
1. Clean text
2. Bag of Words
3. LDA model (Latent Dirichlet allocation)
4. Fine tune LDA model
5. Define Topics from LDA model
6. Create df of document probabilities
6. Nearest Neighbors Model

---

**Import Necessary Libraries**

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from dreamjobber_web.recommend import input_user_scores, make_recommendation, collect_score_and_recommend, collect_feedback, show_to_user
from functions import lemmatize_stem, preprocess, remove_brackets, remove_punctuation, remove_stop_words
from lda import show_topics_sentences
import pickle

#lda model evaluation with coherence
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from sklearn.neighbors import NearestNeighbors

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

**Load Data**

In [3]:
df_1 = pd.read_json('data/dice_jobs_1.json', lines=True)
df_2 = pd.read_json('data/dice_jobs_2.json', lines=True)
df_3 = pd.read_json('data/dice_jobs_3.json', lines=True)
df_4 = pd.read_json('data/dice_jobs_4.json', lines=True)
df_5 = pd.read_json('data/dice_jobs_5.json', lines=True)
df_6 = pd.read_json('data/dice_jobs_6.json', lines=True)

In [4]:
#concat into one df
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True, sort=True)

In [5]:
df.head()  

,job_description,job_title
0,NaN,UI Lead/Architect
1,NaN,Web Application Architect
2,NaN,Senior DataStage Developer
3,NaN,Hadoop Administrator
4,NaN,UX Visual Designer


In [6]:
#check for missing values
df.isna().sum()

job_description    6524
job_title          5484
dtype: int64

In [7]:
#looks like there are rows that have no job description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27016 entries, 0 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          21532 non-null object
dtypes: object(2)
memory usage: 422.2+ KB


In [8]:
#drop rows with no job descriptions
df = df.dropna()

In [9]:
#sanity check, looks good
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [10]:
df.head()

,job_description,job_title
9,[2+ years of experience developing Java / J2EE...,Java Developer (Sign-On BONUS!)
10,"[Passion for technology and learning, a natura...","iOS Developer - Mobile Rate: Open, Duration: 1..."
17,[We enjoy approved IT vendor status with sever...,"EUC Engineer, Rate-Open, Duration: 18 Months"
18,[We enjoy approved IT vendor status with sever...,"Software Developer - RPG, Rate-Open, Duration:..."
19,[SME in Linux Operating system with Strong Vir...,Sr. Linux Consultant with Weblogic exp


In [11]:
#need to remove brackets from job_description
df['job_description'] = df['job_description'].map(remove_brackets)

In [12]:
#remove '\\n' and replace with ','
df['job_description'] = df['job_description'].map(lambda x: x.replace('\\n', ','))

In [13]:
#lowercase job_description text before applying stopwords
df['job_description'] = df['job_description'].map(lambda x: x.lower())

In [14]:
#lowercase job_title text before cleaning
df['job_title'] = df['job_title'].map(lambda x: x.lower())

In [15]:
#remove punctuation from job_title
df['job_title'] = df['job_title'].map(remove_punctuation)

In [16]:
#remove stop words from job_title
df['job_title'] = df['job_title'].map(remove_stop_words)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [18]:
#drop any duplicates
df = df.drop_duplicates()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18704 entries, 9 to 27015
Data columns (total 2 columns):
job_description    18704 non-null object
job_title          18704 non-null object
dtypes: object(2)
memory usage: 438.4+ KB


In [20]:
#reset the index
df = df.reset_index(drop=True)

In [21]:
#check to see if there are any job titles that have been removed via stopwords, I will want to remove these rows
#because the job titles were not real job titles
df.loc[df['job_title']=='']

,job_description,job_title
1526,'',
2508,'robert half technology is looking for an expe...,
13230,'someone who has spent 8-10 years in insurance...,
13569,"'my client is looking for sap fico in houston,...",
14136,"'share your resume on click here to apply', 'j...",
14157,"'role : fico with s4 hana.,location : manitowo...",
15118,"'hello all,', '', '', '', '', '', '', '', '', ...",
16508,"'job description:', 'leidos is looking for a j...",


In [22]:
df = df.drop(df.index[[1526, 2508, 13230, 13569, 14136, 14157, 15118, 16508]])

In [23]:
#sanity check
df.loc[df['job_title']=='']

,job_description,job_title


In [24]:
#reset index one last time
df = df.reset_index(drop=True)
df.head()

,job_description,job_title
0,'2+ years of experience developing java / j2ee...,java developer
1,"'passion for technology and learning, a natura...",ios developer mobile
2,'we enjoy approved it vendor status with sever...,euc engineer
3,'we enjoy approved it vendor status with sever...,software developer rpg
4,'sme in linux operating system with strong vir...,linux consultant with weblogic exp


---

## Text Cleaning

1. Tokenize
2. Remove words with fewer than 2 characters
3. Remove stop words
4. Normalize words (Lemmatize and Stem)

**Test the functions on one row of text**

In [25]:
stemmer = SnowballStemmer('english')

In [26]:
text_sample = df[df.index == 13].values[0][0]

print('original text: ')
words = []
for word in text_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized text: ')
print(preprocess(text_sample))

original text: 
["'demonstrates", 'brand', 'passion,champions', 'and', 'embraces', 'change,makes', 'good', 'decisions,delivers', 'results,takes', 'action', 'with', 'integrity,communicates', "effectively'"]


 tokenized and lemmatized text: 
['demonstr', 'brand', 'passion', 'champion', 'embrac', 'chang', 'make', 'good', 'decis', 'deliv', 'result', 'take', 'action', 'integr', 'communic', 'effect']


**Apply cleaning functions to job_description**

In [ ]:
#apply text cleaning function and display first 5 rows
processed_text = df['job_description'].map(preprocess)
processed_text[:5]

---

## Bag of Words

In [ ]:
#I'll use bag of words to extract features from text for use in modeling

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_text)

In [ ]:
#check the length before I filter out the extremes
len(dictionary)

In [ ]:
dictionary.filter_extremes(no_below=25, no_above=0.5, keep_n=100000)

In [ ]:
#check length after filtering out extremes
len(dictionary)

In [ ]:
#bow2doc: counts the number of occurrences of each distinct word, 
#converts the word to its integer word id and returns the result as a sparse vector

bow2doc_corpus = [dictionary.doc2bow(text) for text in processed_text]

---

## LDA model with Bag of Words

In [ ]:
lda_model = gensim.models.LdaMulticore(bow2doc_corpus, 
                                       num_topics=9, 
                                       id2word=dictionary, 
                                       passes=50, 
                                       workers=4,
                                      chunksize=500)


In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

**Pickled LDA model results**

In [ ]:
#pickle.dump(lda_model, open('dreamjobber_web/webapp/pickled_models/lda_model.pkl', 'wb'))
pickled_lda = pickle.load(open('dreamjobber_web/webapp/pickled_models/lda_model.pkl', 'rb'))

In [ ]:
for idx, topic in pickled_lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
#manually name topics
col_names=['Analyst', 'Security', 'Leadership', 'Software/Web Dev', 
           'Cloud Computing', 'Computer Network', 'Database Admin', 'Computer Support', 'WebDev']



---

**LDA model evaluation**

In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Compute Coherence Score using UMass
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_text, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
##visualize the topics in order to better label 
#you may need to pip install pyLDAvis
#!pip install pyLDAvis
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=bow2doc_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


In [ ]:
#show topics and descriptions
df_topic_sents_keywords = show_topics_sentences(ldamodel=pickled_lda, corpus=bow2doc_corpus, texts=df['job_description'])


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [ ]:
df_dominant_topic.head()

---

**Create df for topic scores for each jobtitle**

In [ ]:
#get probabilities of a document belonging to each topic and append to list
topic_vecs = []
for i in range(len(bow2doc_corpus)):
    top_topics = pickled_lda.get_document_topics(bow2doc_corpus[i], minimum_probability=0.0)
    #i in range(amount of topics)
    topic_vec = [top_topics[i][1] for i in range(9)]
    topic_vecs.append(topic_vec)

In [ ]:
#create a dataframe for topic scores
df_topic_vecs = pd.DataFrame(topic_vecs)
df_topic_vecs.head(10)

In [ ]:
#name columns for dataframe

col_names=['Analyst', 'Security', 'Leadership', 'Software/Web Dev', 
           'Cloud Computing', 'Computer Network', 'Database Admin', 'Computer Support', 'WebDev']



df_topic_vecs.columns = col_names
df_topic_vecs.head()

---

# Nearest Neighbors

In [ ]:
#next step merge with original df of job titles and job descriptions
#pickle the merged df

In [ ]:
df_final = pd.merge(df, df_topic_vecs,left_index=True, right_index=True)

In [ ]:
#pickle.dump(df_final, open('dreamjobber_web/webapp/pickled_models/df_final.pkl', 'wb'))
pickled_df_final = pickle.load(open('dreamjobber_web/webapp/pickled_models/df_final.pkl', 'rb'))
pickled_df_final.head()

In [ ]:
topics = pickled_df_final.drop(['job_description', 'job_title'], axis=1)
jobs = pickled_df_final['job_title']

In [27]:
#pickle jobs for use in webapp
#pickle.dump(jobs, open('dreamjobber_web/webapp/pickled_models/jobs.pkl', 'wb'))
jobs = pickle.load(open('dreamjobber_web/webapp/pickled_models/jobs.pkl', 'rb'))

In [ ]:
nearest_neighbor = NearestNeighbors(n_neighbors=50)
nearest_neighbor.fit(topics)

In [28]:
#pickle.dump(nearest_neighbor, open('dreamjobber_web/webapp/pickled_models/nn_model.pkl', 'wb'))
nearest_neighbor = pickle.load(open('dreamjobber_web/webapp/pickled_models/nn_model.pkl', 'rb'))

---

**Make Recommendations**

In [29]:
show_to_user(nearest_neighbor, jobs)


Scale of 0-10.
    0 is Do NOT agree and 10 is agree
Agree or Disagree: I am/I like Analyst: 1
Agree or Disagree: I am/I like Security: 2
Agree or Disagree: I am/I like Leadership: 3
Agree or Disagree: I am/I like Software/Web Dev: 4
Agree or Disagree: I am/I like Cloud Computing: 5
Agree or Disagree: I am/I like Computer Network: 6
Agree or Disagree: I am/I like Database Admin: 7
Agree or Disagree: I am/I like Computer Support: 8
Agree or Disagree: I am/I like WebDev: 9


['1. firmware engineer iot engineer', '2. media buyer planner', '3. academic instructional designer', '4. digital content writer', '5. marketing manager product developer', '6. studio artist', '7. data analyst', '8. business system analyst with netsuite consultant', '9. artist rigger', '10. tech artist']
How did you like your recommendations? bad, okay, or goodbad
